In [2]:
from metric_calculator import calculate_metrics
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [7]:
### load sample data
with h5py.File("data/sample_data.h5") as hf:
    gt_fids = hf["ground_truth_fids"][()]
    ppm = hf["ppm"][()]
    t = hf["t"][()]

# transforming into frequency domain to use as the target data
gt_spec = np.fft.fftshift(np.fft.ifft(gt_fids,axis=1),axes=1)

y = np.real(gt_spec[:,:,1]-gt_spec[:,:,0])

# setting up data to desired format
y_max = y.max(axis=(1),keepdims=True)
y_mean = y[:,900:1100].min(axis=(1),keepdims=True)
y = (y-y_mean)/(y_max-y_mean)
y_test = y[int(y.shape[0]*0.8):]

# loading data from other submissions
submission_labels = ["tf","torch"]
submissions = []

for i in range(len(submission_labels)):
    with h5py.File(f"Tutorials/sample_submissions/{submission_labels[i]}_submission.h5") as hf:
        i_spectra = hf["result_spectra"][()]
        i_ppm = hf["ppm"][()]
        submissions.append({"label":submission_labels[i],"spec":i_spectra,"ppm":i_ppm})


In [13]:
metric_results = {}

for item in submissions:
    metric_results[item['label']] = calculate_metrics(item["spec"],y_test,item["ppm"])

#print(metric_results)
print(f"Tensorflow Sample Model Metrics: {' - '.join(['{}:{:.2e}'.format(key,value) for key,value in metric_results['tf'].items()])}")
print(f"---Pytorch Sample Model Metrics: {' - '.join(['{}:{:.2e}'.format(key,value) for key,value in metric_results['torch'].items()])}")

Tensorflow Sample Model Metrics: mse:4.03e-04 - snr:9.74e+01 - linewidth:9.44e-02 - shape_score:9.39e-01
---Pytorch Sample Model Metrics: mse:7.06e-03 - snr:6.93e+01 - linewidth:9.58e-02 - shape_score:9.49e-01
